# Lab 6:  EDA using a massive airline dataset

For this lab, you'll need to access a 500MB+ dataset at kaggle.com.  First, register at kaggle.com, login and download the dataset at https://www.kaggle.com/usdot/flight-delays.  Then, visit https://drive.google.com/drive/my-drive and find a place to upload this dataset (using the "New" button).  I have created a directory called "Colab Notebooks/data" and uploaded the files there.

The first step, as usual, is to load the necessary python resources:

In [0]:
# Load needed python resources
from datascience import *
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np
np.set_printoptions(threshold=50)


Next we'll read in the massive dataset, which has been stored as a csv file in a directory I created on my google drive space.  

In [0]:
flights = Table.read_table('drive/My Drive/Colab Notebooks/data/flights.csv')

Let's check out the columns available in the flights dataset:

In [0]:
flights.labels

Next, select a systematic sample of rows from the flights Table object using code similar to the Chapter 10 intro.  You'll need to figure out an appropriate value of 'gap' based on the lab instructions.

In [0]:
gap = **ENTER AN APPROPRIATE INTEGER VALUE OF gap HERE**
start = np.random.choice(np.arange(gap))
mySample = flights.take(np.arange(start, flights.num_rows, gap))
print(mySample.num_rows)
mySample

Load the airports dataset containing airport names and lat/lon coordinates:


In [0]:
airports = Table.read_table('drive/My Drive/Colab Notebooks/data/airports.csv')

String together multiple Table-modifying methods (from the datascience module) to produce a cleaned-up version of the systematic sample called mySample (then print its number of rows and its first 10 rows):



1.   Select mySample according to 'gap' and 'start'.
2.   Then join every ORIGIN_AIRPORT in 'mysample' with the
 corresponding columns from 'airports' based on matching IATA_CODE.
3.   Then select just the columns we need from the result.
4.   Then relabel some of the columns.


In [0]:
mySample = (flights.take(np.arange(start, flights.num_rows, gap))
                   .join('ORIGIN_AIRPORT', airports, 'IATA_CODE')
                   .select('MONTH', 'DAY', 'ORIGIN_AIRPORT', 
                           'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 
                           'DEPARTURE_DELAY', 'AIRPORT',
                           'LATITUDE', 'LONGITUDE')
                   .relabeled('ORIGIN_AIRPORT', 'ORIGIN')
                   .relabeled('DESTINATION_AIRPORT', 'DESTINATION')
                   .relabeled('DEPARTURE_DELAY', 'DELAY')
                   .relabeled('AIRPORT', 'ORIGIN_NAME')
            )
print(mySample.num_rows)
mySample

Create a new column with the approximate day of year. There are better, more accurate ways to do this, but this method that approximates each month by 30 days will work for this purpose:

In [0]:
mySample = mySample.with_column(
             'APPROX_DAY_OF_YEAR', 
             30*(mySample.column('MONTH')-1) + mySample.column('DAY'))
mySample

Create a scatterplot using the 'LATITUDE' and 'DELAY' columns.  Please notice that a scatterplot depicts the relationship between two quantitative measurement columns:


In [0]:
mySample.scatter('LATITUDE', 'DELAY')

# EXTRA CODE
The code below is not strictly necessary for the lab assignment.  It is included to illustrate some potentially interesting directions you could take your own investigation:

In [0]:
# Create a histogram of LATITUDE
mySample.hist('LATITUDE')

# Create a new column that splits the airports into high vs. low latitude
# based on a cutoff you define:
LatCut = **ENTER A VALUE THAT MAKES SENSE HERE**
mySample = mySample.with_column(
              'HIGH_LAT', mySample.column('LATITUDE') > LatCut)
mySample


# Figure out how many airports are "high latitude" vs. "low latitude"
mySample.group('HIGH_LAT')


# Take the means for the high- and low- latitude airports
# We're using the 'nanmean' method in numpy to ignore the 
# nan (not a number) values
mySample.group('HIGH_LAT', np.nanmean)


# Use the previous idea to find the mean difference automatically
Observed_mean_difference = np.diff(mySample.group('HIGH_LAT', np.nanmean)
                                           .column('DELAY nanmean'))[0]
Observed_mean_difference


# Define a function that will reshuffle the DELAY values and then return
# the mean difference statistic for the shuffled table.
# This simulates from the null hypothesis distribution of the 
# mean difference statistic.
def simulated_mean_difference_under_null():
    a=(mySample.sample(with_replacement=False)
               .select('DELAY')
               .with_column('HIGH_LAT', mySample.column('HIGH_LAT')))
    return (np.diff(a.group('HIGH_LAT', np.nanmean)
                     .column('DELAY nanmean'))[0])

                     
# Simulate 5000 draws from the null hypothesis distribution of the 
# mean difference (and return the result as a numpy array--not the same 
# as a datascience Table)
H0_means = make_array()
for i in np.arange(5000):
    H0_means = np.append(H0_means, simulated_mean_difference_under_null())    
                     

# Create a table with the 5000 H0 (null hypothesis) values and then 
# create a histogram.
# Also add the observed value of the sample statistic as a red dot 
# along the x-axis.
Table().with_column(
    'Count in a Random Sample', H0_means
).hist(bins = np.arange(-12.5, 12.5, 1))
plots.scatter(Observed_mean_difference, 0, color='red', s=30);
